In [1]:
import pandas as pd
import datetime as dt
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
bets = pd.read_csv('past_bets/base/bets_20220527.csv', index_col = 0)

In [3]:
bets = bets[bets.Home_Team != 'Los Angeles Angels']
bets.reset_index(drop = True, inplace = True)
bets

,Home_Team,Away_Team,Home_Prob,Away_Prob,Home_ML,Away_ML,Home_ML_Prob,Away_ML_Prob,Home_Diff,Away_Diff,Home_KC,Away_KC,Home_Bet,Away_Bet
0,Washington Nationals,Colorado Rockies,0.516414,0.483586,-116.0,-102.0,0.537037,0.504950,-0.020623,-0.021364,0.000000,0.000000,0.000000,0.000000
1,Boston Red Sox,Baltimore Orioles,0.621097,0.378903,-180.0,152.0,0.642857,0.396825,-0.021760,-0.017922,0.000000,0.000000,0.000000,0.000000
2,Detroit Tigers,Cleveland Guardians,0.335453,0.664547,134.0,-158.0,0.427350,0.612403,-0.091898,0.052144,0.000000,0.013453,0.000000,1381.092192
3,New York Mets,Philadelphia Phillies,0.602061,0.397939,-158.0,134.0,0.612403,0.427350,-0.010342,-0.029412,0.000000,0.000000,0.000000,0.000000
4,Tampa Bay Rays,New York Yankees,0.497977,0.502023,-126.0,108.0,0.557522,0.480769,-0.059545,0.021254,0.000000,0.004093,0.000000,420.212817
5,Atlanta Braves,Miami Marlins,0.604712,0.395288,-148.0,126.0,0.596774,0.442478,0.007938,-0.047190,0.001969,0.000000,202.098861,0.000000
6,Minnesota Twins,Kansas City Royals,0.650174,0.349826,-184.0,154.0,0.647887,0.393701,0.002287,-0.043875,0.000649,0.000000,66.667892,0.000000
7,St. Louis Cardinals,Milwaukee Brewers,0.441175,0.558825,122.0,-144.0,0.450450,0.590164,-0.009275,-0.031339,0.000000,0.000000,0.000000,0.000000
8,Arizona Diamondbacks,Los Angeles Dodgers,0.345170,0.654830,132.0,-156.0,0.431034,0.609375,-0.085864,0.045455,0.000000,0.011636,0.000000,1194.586015
9,San Diego Padres,Pittsburgh Pirates,0.718206,0.281794,-220.0,184.0,0.687500,0.352113,0.030706,-0.070318,0.009826,0.000000,1008.710154,0.000000


In [4]:
bets.to_csv('past_bets/base/bets_20220527.csv')